Hey Catherine! This is the commented notebook. I copied the baseline from someone in the discussion panel. But I added comment on everything.
Hey Victor! Great work! I saw some references but do you have the reference to the original baseline code?

# RSNA - EfficientNet Version 0.1 [TF]
_____

### EfficientNet

EfficientNet is one of the most solid baselines known today. It is a family of convolutional neural networks that have achieved state-of-the-art accuracy on ImageNet while also being smaller and faster than other models.
The main idea of EfficientNet is scaling up CNNs in a principled way. It uses a scalable architecture, named compound scaling, which balances network depth, width, and resolution to achieve superior performance.
The image below shows the scaling method in more detail.

![](https://i.ibb.co/Y86KGDg/image4-1.png)

#### Model Size (B5)

As written above, the main claim of efficientnet is providing a method for scaling up neural networks. 
Using this approach, the authors of the paper released the official efficientnet architecture scaled to various sizes [B0, B1, B2.. B7. And two monstrosities L1 and L2].
As it is usually empirically the case (Also hinted by the [Scaling Laws for Neural Language Models](https://arxiv.org/abs/2001.08361)): Performance increase with model size. 
On this notebook we use the **B5** variant of efficientnet. 
Feel free to experiment with larger sizes.

**Installations [Hidden cell]**

In [1]:
import sys
sys.path.append('../input/kerasapplications')
sys.path.append('../input/efficientnet-keras-source-code/')
import keras_applications
import efficientnet.tfkeras as efficientnet

# This cell imports the model "Efficientnet", which is a pretrained model
# We are doing transfer learning for now

### Imports

In [2]:
import os
import cv2
import glob
import traceback
import cv2 as cv
import numpy as np
import pandas as pd
from path import Path
from tqdm import tqdm
import nibabel as nib
import pydicom as dicom
import tensorflow as tf
from keras import layers
from pydicom import dcmread
from tensorflow import keras
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from pydicom.data import get_testdata_files
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array

### Data Loading

> **Note:** Mean target calculation for a baseline submission.

I'm printing the important stuff but we'll come to it later. When we create the submission. 
For now we are just loading the csv.

In [3]:
# All this chunck of code is to prepare the test for the prediction/submission
bad = np.array([['1.2.826.0.1.3680043.10197_C1', '1.2.826.0.1.3680043.10197','C1'],['1.2.826.0.1.3680043.10454_C1', '1.2.826.0.1.3680043.10454','C1'],['1.2.826.0.1.3680043.10690_C1', '1.2.826.0.1.3680043.10690','C1']], dtype=np.object)

train_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
test_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/test.csv")

train_dir = '../input/rsna-2022-cervical-spine-fracture-detection/train_images'
test_dir = '../input/rsna-2022-cervical-spine-fracture-detection/test_images'
first_image = os.path.join(test_dir, test_df['StudyInstanceUID'].iloc[0])

new_submission = []
means = train_df.median(numeric_only=True).to_dict()
means = dict(zip(train_df.columns[1:], np.average(train_df[train_df.columns[1:]], axis=0, weights=train_df["patient_overall"] + 1)))
prediction_type = test_df['prediction_type'].tolist()
submission = pd.read_csv('../input/rsna-2022-cervical-spine-fracture-detection/sample_submission.csv')
for i in range(len(submission)):        
    new_submission.append(means[prediction_type[i]])
submission['fractured'] = new_submission


if(test_df.values[0][0] == bad[0][0]): test_df = pd.DataFrame({"row_id": ['1.2.826.0.1.3680043.22327_C1', '1.2.826.0.1.3680043.25399_C1', '1.2.826.0.1.3680043.5876_C1'], "StudyInstanceUID": ['1.2.826.0.1.3680043.22327', '1.2.826.0.1.3680043.25399', '1.2.826.0.1.3680043.5876'], "prediction_type": ["C1", "C1", "C1"]})  
prediction_type_mapping = test_df['prediction_type'].map({'C1': 0, 'C2': 1, 'C3': 2, 'C4': 3, 'C5': 4, 'C6': 5, 'C7': 6}).values

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [4]:
test_df

,row_id,StudyInstanceUID,prediction_type
0,1.2.826.0.1.3680043.22327_C1,1.2.826.0.1.3680043.22327,C1
1,1.2.826.0.1.3680043.25399_C1,1.2.826.0.1.3680043.25399,C1
2,1.2.826.0.1.3680043.5876_C1,1.2.826.0.1.3680043.5876,C1


In [5]:
bad

array([['1.2.826.0.1.3680043.10197_C1', '1.2.826.0.1.3680043.10197',
        'C1'],
       ['1.2.826.0.1.3680043.10454_C1', '1.2.826.0.1.3680043.10454',
        'C1'],
       ['1.2.826.0.1.3680043.10690_C1', '1.2.826.0.1.3680043.10690',
        'C1']], dtype=object)

In [6]:
submission

,row_id,fractured
0,1.2.826.0.1.3680043.10197_C1,0.097987
1,1.2.826.0.1.3680043.10454_C1,0.097987
2,1.2.826.0.1.3680043.10690_C1,0.097987


In [7]:
new_submission

[0.09798657718120805, 0.09798657718120805, 0.09798657718120805]

In [8]:
prediction_type_mapping

array([0, 0, 0])

In [9]:
test_df

,row_id,StudyInstanceUID,prediction_type
0,1.2.826.0.1.3680043.22327_C1,1.2.826.0.1.3680043.22327,C1
1,1.2.826.0.1.3680043.25399_C1,1.2.826.0.1.3680043.25399,C1
2,1.2.826.0.1.3680043.5876_C1,1.2.826.0.1.3680043.5876,C1


#### Utility Functions & Variables

- **load_dicom:** For loading (and converting) a single dicom image.
- **lisdirs:** For listing the data directory.
- **train_dir / test_dir:** used later on for fast switching between train & validation paths

In [10]:
def load_dicom(path, size = 64):
    try:
        # Read the dicom image
        img=dicom.dcmread(path)
        img.PhotometricInterpretation = 'YBR_FULL'
        # extract the array
        data=img.pixel_array
        #Standarisation
        data=data-np.min(data)
        if np.max(data) != 0:
            data=data/np.max(data)
        # This is augmented to scale 0-255 for easier manipulation
        data=(data*255).astype(np.uint8)
        # Change to gray scale
        return cv2.cvtColor(data.reshape(512, 512), cv2.COLOR_GRAY2RGB)
    except:        
        # If there is an error with the image, it returns an empty img
        return np.zeros((512, 512, 3))

def listdirs(folder):
    return [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]    

train_dir = '../input/rsna-2022-cervical-spine-fracture-detection/train_images'
test_dir = '../input/rsna-2022-cervical-spine-fracture-detection/test_images'
patients = sorted(os.listdir(train_dir))

#### Quick Visualizations

> **Credit:** This great [notebook](https://www.kaggle.com/code/realneuralnetwork/rsna-efficientnet-infer) by [Kabir Ivan](https://www.kaggle.com/realneuralnetwork)

Simple sanity test - mainly for testing out the load_dicom function.load_dicom

Uncomment to visualize

**Training Samples**

In [11]:
# image_file = glob.glob("../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.10001/*.dcm")
# plt.figure(figsize=(20, 20))

# for i in range(28):
#     ax = plt.subplot(7, 7, i + 1)
#     image_path = image_file[i]
#     image = load_dicom(image_path)
#     plt.axis('off')   
#     plt.imshow(image)

**Targets**

In [12]:
# image_file = glob.glob("../input/rsna-2022-cervical-spine-fracture-detection/segmentations/*.nii")
# plt.figure(figsize=(20, 20))

# for i in range(28):
#     ax = plt.subplot(7, 7, i + 1)
#     image_path = image_file[i]
#     nii_img = nib.load(image_path).get_fdata()
#     nib_image = nii_img[:,:,59]
#     plt.axis('off')
#     plt.imshow(nib_image)

### Data Generators

Since the training data can be a bit large, we load it batch by batch when training & inference.
This is done through simple keras generators. 

#### Train Generator 

- Also yields labels

In [13]:
def RSNATrainGenerator(train_df, batch_size, infinite = True, base_path = train_dir):
    while True: # this will be running during the whole training.
        # Initialize the sets
        trainset = []
        trainidt = []
        trainlabel = []
        for i in (range(len(train_df))): # Thjs can be considered the epoch lenght
            idt = train_df.loc[i, 'StudyInstanceUID'] # Select one patient at time
            path = os.path.join(train_dir, idt)
            for im in os.listdir(path): # Iterate over the images
                dc = dicom.read_file(os.path.join(path,im))
                if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
                    continue
                img = load_dicom(os.path.join(path , im)) #Load DICOM
                img = cv.resize(img, (128 , 128))         # Resize
                image = img_to_array(img)                 # Keras util to extract array from image
                image = image / 255.0                     # Standarizing values to [0,1]
                trainset += [image]                       # Appending to list
                cur_label = []                            # Adding labels
                cur_label.append(train_df.loc[i,'C1'])
                cur_label.append(train_df.loc[i,'C2'])
                cur_label.append(train_df.loc[i,'C3'])
                cur_label.append(train_df.loc[i,'C4'])
                cur_label.append(train_df.loc[i,'C5'])
                cur_label.append(train_df.loc[i,'C6'])
                cur_label.append(train_df.loc[i,'C7'])
                trainlabel += [cur_label]
                trainidt += [idt]
                if len(trainidt) == batch_size:          # If a batch is completed, yield and restart          
                    yield np.array(trainset), np.array(trainlabel) # yield is a keyword like "return" but it doesn't stop the function
                    trainset, trainlabel, trainidt = [], [], []
            i+=1

#### Test Generator

- Yields only image samples

In [14]:
#Same with tests
def RSNATestGenerator(test_df, batch_size, infinite = True, base_path = test_dir):
    while 1:        
        testset=[]
        testidt=[]
        for i in (range(len(test_df))):        
            if type(test_df) is list: idt = test_df[i]
            else: idt = test_df['StudyInstanceUID'].iloc[i]
            path = os.path.join(base_path, idt)
            if os.path.exists(path):
                for im in os.listdir(path):
                    dc = dicom.read_file(os.path.join(path,im))
                    if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
                        continue
                    img=load_dicom(os.path.join(path,im))
                    img=cv.resize(img,(128, 128))
                    image=img_to_array(img)
                    image=image/255.0
                    testset+=[image]
                    testidt+=[idt]
                    if len(testset) == batch_size:                        
                        yield np.array(testset)
                        testset = []
        if len(testset) > 0: yield np.array(testset)
        if not infinite: break

#### The Model

> **Note:** The training data loads as a single channel image, we use a simple `Conv2D` to align it to be a 3 channel image (As expected by efficientnet).

In [15]:
def get_model():
    # Define the input. Here is None for the transfer learning stuff. Check the Tensorflow tutorial to know more
    inp = keras.layers.Input((None, None ,1))
    # First Conv2D to extract the info
    x = Conv2D(3, 64, padding = 'SAME')(inp)
    #The pretrained model
    x = efficientnet.EfficientNetB6(include_top=False, weights='../input/efficientnet-keras-dataset/weights/efficientnet-b6_noisy-student_notop.h5')(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    # The output layer returns a probability for each vertebra. 
    out = keras.layers.Dense(7, 'sigmoid')(x)
    # The next three lines are putting the model togheter
    model = keras.models.Model(inp, out)
    model.summary()
    model.compile(loss="binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001))
    return model

#### Main Training Cell

We use a stratified (based on patient_overall) KFold split and train 5 different models. 

In [16]:
# Defining callbacks. Special instructions during training
Early = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
callbacks = [
    Early    
]

In [17]:
# # for train_idx, val_idx in StratifiedKFold(5).split(train_df, train_df['patient_overall']):    
# #     K.clear_session()
# Defining train and validation sets. 80%-20%
x_train = train_df.iloc[:1616].reset_index()
x_val = train_df.iloc[1616:].reset_index()
model = get_model()
# fitting the model with the data yield from the generator
hist = model.fit_generator(                            
                                RSNATrainGenerator(x_train, min(len(x_train), 64), infinite = False, base_path = train_dir),
                                epochs = 50,
                                verbose = 1,
                                callbacks = [callbacks],
                                validation_steps = max((len(x_val) // 64), 1),
                                steps_per_epoch = max((len(x_train) // 64), 1),
                                validation_data = RSNATrainGenerator(x_val, min(len(x_val), 64), infinite = False, base_path = train_dir),
                          )

# Here, we do the same with the validation to check model validation loss
val_pred = model.predict_generator(RSNATestGenerator(x_val, min(len(test_df), 64), infinite = False, base_path = train_dir), steps = max((len(test_df) // 64), 1))  

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 3)     12291     
_________________________________________________________________
efficientnet-b6 (Functional) (None, None, None, 2304)  40960136  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 16135     
Total params: 40,988,562
Trainable params: 40,764,130
Non-trainable params: 224,432
_________________________________________________________________


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
25/25 [==============================] - 85s 2s/step - loss: 0.6293 - val_loss: 0.6247
Epoch 2/50
25/25 [==============================] - 43s 2s/step - loss: 0.3184 - val_loss: 0.1361
Epoch 3/50
25/25 [==============================] - 43s 2s/step - loss: 0.3870 - val_loss: 0.0737
Epoch 4/50
25/25 [==============================] - 45s 2s/step - loss: 0.4695 - val_loss: 0.1413
Epoch 5/50
25/25 [==============================] - 47s 2s/step - loss: 0.7186 - val_loss: 0.4719
Epoch 6/50
25/25 [==============================] - 55s 2s/step - loss: 0.4370 - val_loss: 0.3446
Epoch 7/50
25/25 [==============================] - 46s 2s/step - loss: 0.3492 - val_loss: 0.1865
Epoch 8/50
25/25 [==============================] - 52s 2s/step - loss: 0.2082 - val_loss: 0.3250


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [18]:
try: # predictions
    preds = model.predict_generator(RSNATestGenerator(test_df, min(len(test_df), 64), infinite = False, base_path = test_dir), steps = max((len(test_df) // 64), 1))

    new_preds = []
    for pred_idx in range(len(preds)):
        new_preds.append(preds[pred_idx][prediction_type_mapping[pred_idx]])
    # submission['fractured'] += preds[:, prediction_type_mapping] / 5
    submission['fractured'] += np.array(new_preds) / 5

except: traceback.print_exc()    

In [19]:
#submission

#### Submission

In [20]:
submission.to_csv('submission.csv', index = 0)